In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import random
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 预授权人数

In [29]:
# sql_ysqfm = '''
#     select DATE_FORMAT(create_time,'%Y-%m-%d'), count(*) 
#     from db_digua_business.t_order_pay 
#     where pay_type = 'ZFBYSQ' 
#     -- and  DATE_FORMAT(create_time, '%Y-%m-%d')>='2025-01-01'
#     -- and  DATE_FORMAT(create_time, '%Y-%m-%d')<='2025-08-12'
#     and  create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )
#     group by DATE_FORMAT(create_time,'%Y-%m-%d') -- 分母
#     '''
    
sql_ysq = '''
    select om.create_time, tmu.id_card_num,top.order_id,om.status
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
    from db_digua_business.t_order_pay top
    left join db_digua_business.t_order om on top.order_id = om.id
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
    left join db_digua_business.t_order_risk tor on om.id = tor.order_id 
    where top.pay_type = 'ZFBYSQ' 
    and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-08-01'
    and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-08-19'
    -- and  create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )
    and om.user_mobile is not null 
    and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')   
    -- and top.total_freeze_fund_amount is not null
    -- and trade_no is not null
    '''
    
df_ysq = clean.query(sql_ysq)
    
df_ysq.shape

(100246, 5)

In [30]:
df = df_ysq.copy()
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df.loc[:,"进件"]=np.where((df["status"]==1)|(df["status"]==13),"未进件","进件")
# 去除是否前置拦截=1且进件=未进件的行
df_ans = df[(df['是否前置拦截']==0)|(df['进件']=='进件')]
# 按下单日期与id_card_num去重
df_ans = df_ans.drop_duplicates(subset=["下单日期","id_card_num"],keep="first")
# 按下单日期分组计算订单数
df_ans = df_ans.groupby(["下单日期"]).agg({"order_id": 'count'}).rename(columns={"order_id": "预授权调起人数"}).reset_index()
df_ans

,下单日期,预授权调起人数
0,2025-08-01,2775
1,2025-08-02,2341
2,2025-08-03,2209
3,2025-08-04,2882
4,2025-08-05,3096
5,2025-08-06,3070
6,2025-08-07,3088
7,2025-08-08,3060
8,2025-08-09,2610
9,2025-08-10,2316


## 预授权成功人数

In [24]:
sql_ysq_success = '''
    select om.create_time, tmu.id_card_num,top.order_id
    from db_digua_business.t_order_pay top
    left join db_digua_business.t_order om on top.order_id = om.id
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id 
    where top.pay_type = 'ZFBYSQ' 
    and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-08-01'
    and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-08-19'
    -- and  create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )
    and om.user_mobile is not null 
    and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')   
    and top.total_freeze_fund_amount is not null
    and trade_no is not null
    and top.status in (2, 5)
    '''
df_ysq_success = clean.query(sql_ysq_success)
df_ysq_success.shape
    

(22781, 3)

In [28]:
df_success = sql_ysq_success.copy()
df_success["下单日期"]=df_success["create_time"].dt.date
df_success["下单日期"]=pd.to_datetime(df_success["下单日期"],errors="coerce")
# 按下单日期降序排序
# df_success = df_success.sort_values(by="下单日期",ascending=False)
# df_success.loc[:, "去重逻辑"] = df_success["下单日期"].astype(str)+df_success["id_card_num"]
# 按下单日期与id_card_num去重
df_success = df_success.drop_duplicates(subset=["下单日期","id_card_num"],keep="first")
# df_success = df_success.drop_duplicates(subset=["去重逻辑"],keep="first")
# 按下单日期分组计算订单数
df_success = df_success.groupby(["下单日期"]).agg({"order_id": 'count'}).rename(columns={"order_id": "预授权成功人数"}).reset_index()
df_success#.order_id.sum()

,下单日期,预授权成功人数
0,2025-08-01,1195
1,2025-08-02,1062
2,2025-08-03,1017
3,2025-08-04,1307
4,2025-08-05,1322
5,2025-08-06,1304
6,2025-08-07,1284
7,2025-08-08,1302
8,2025-08-09,1214
9,2025-08-10,1039


In [31]:
# 根据下单日期合并两DataFrame：df_success，df_ans
df_res = pd.merge(df_success, df_ans, on=['下单日期'], how='left')
df_res

,下单日期,预授权成功人数,预授权调起人数
0,2025-08-01,1195,2775
1,2025-08-02,1062,2341
2,2025-08-03,1017,2209
3,2025-08-04,1307,2882
4,2025-08-05,1322,3096
5,2025-08-06,1304,3070
6,2025-08-07,1284,3088
7,2025-08-08,1302,3060
8,2025-08-09,1214,2610
9,2025-08-10,1039,2316


# 读取数据

In [36]:
sql1 = '''  
SELECT om.id as order_id ,om.order_number,om.all_money 总租金, om.create_time
,om.status, om.user_mobile
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_id 商品id
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_name
,pa.type
,tos.sort, tos.money, tos.real_pay_money
,om.order_method, om.order_type, tmu.id_card_num, tmu.true_name, tolog.delivery_province_name,
tolog.delivery_city_name, tolog.delivery_county_name, top.total_describes
from  db_digua_business.t_order  om
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 订单物流表
left join db_digua_business.t_order_logistics tolog on tolog.order_id=om.id
-- 分期表
left join db_digua_business.t_order_stages tos on om.id = tos.order_id
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')

and om.create_time BETWEEN (CURDATE() - INTERVAL 1 DAY) + INTERVAL 18 HOUR 
                    AND CURDATE() + INTERVAL 18 HOUR               -- 当天数据

'''

df_order = clean.query(sql1)
df_order.shape

(167274, 26)

In [39]:
df = df_order.copy()
df["下单日期"]=df["create_time"]#.dt.date
# df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)
# 删除重复单号
df.drop_duplicates(subset=["order_id"], inplace=True)
# 删除身份证空值行
df.dropna(subset=["id_card_num"], axis=0, inplace=True)
df.drop(df[df['activity_name'] == "1000单秘密计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单秘密计划-无优惠"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单曙光计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "线下门店3个月试行"].index, inplace=True)
df.drop_duplicates(subset=["true_name", "user_mobile", "id_card_num", "下单日期"], keep="last", inplace=True)

In [40]:
df_dy = df[df['归属渠道']=='抖音渠道']
df_dy.shape

(94, 29)

In [41]:
df_dy['create_time']

2640     2025-08-11 18:12:04
3240     2025-08-11 18:14:32
3366     2025-08-11 18:15:05
3468     2025-08-11 18:15:53
5484     2025-08-11 18:25:27
                 ...        
159038   2025-08-12 16:33:09
160058   2025-08-12 16:38:38
160154   2025-08-12 16:39:13
161270   2025-08-12 16:43:18
162713   2025-08-12 16:47:18
Name: create_time, Length: 94, dtype: datetime64[ns]

# 抖音日订单数据定时任务（廖志彧需求）

In [42]:
# 一行代码实现：用 next 取第一个匹配项，无匹配则返回 None
def getvalue(s):
        color_list = json.loads(s)
        for j in range(0, len(color_list)):
            if color_list[j]["key"] == "租赁时长":
                return color_list[j]["value"]

df_dy.loc[:, "租赁时长"] = df_dy.apply(lambda x: getvalue(x["sku_attributes"]),axis=1)
# 获取首期租金：sort=1时的money
df_dy.loc[:, "首期租金"] = np.where(df_dy["sort"]==1,df_dy["money"],0)
# 获取已付租金，real_pay_money的总和
df_dy.loc[:, "已付租金"] = df_dy.groupby(["order_id"])["real_pay_money"].transform(sum)
df_dy.loc[:, "订单状态"] = df_dy["status2"]
df_dy.loc[:, "收货地址"] = df_dy.delivery_province_name + df_dy.delivery_city_name + df_dy.delivery_county_name

dy_dy_res = df_dy[['下单日期', 'order_number' , '商品id', '租赁时长', '总租金','首期租金', '已付租金', '订单状态', 'true_name', '收货地址', 'total_describes']].rename(columns={'order_number': '订单编号', 'true_name': '收货人名称','total_describes': '备注'})

In [43]:
dy_dy_res

,下单日期,订单编号,商品id,租赁时长,总租金,首期租金,已付租金,订单状态,收货人名称,收货地址,备注
2640,2025-08-11 18:12:04,A20250811181204227,8290,365天,5820.0,485.0,0.0,订单取消,苗袁杰,山西省临汾市尧都区,None
3240,2025-08-11 18:14:32,A20250811181432278,8288,180天,3936.0,656.0,0.0,已退款,常佳鑫,广东省广州市海珠区,"[""13760145661订单A20250811181432278，msg=已加微信"", ""..."
3366,2025-08-11 18:15:05,A20250811181504289,8288,180天,3936.0,656.0,0.0,订单取消,常佳鑫,广东省广州市海珠区,None
3468,2025-08-11 18:15:53,A20250811181553298,8288,365天,6900.0,575.0,575.0,待收货,李万辉,福建省宁德市蕉城区,"[""13760145661订单A20250811181553298，msg=加微信"", ""订..."
5484,2025-08-11 18:25:27,A20250811182527476,8288,365天,6156.0,513.0,0.0,订单取消,张垒,湖北省襄阳市樊城区,None
...,...,...,...,...,...,...,...,...,...,...,...
159038,2025-08-12 16:33:09,A20250812163308673,8288,365天,6204.0,517.0,517.0,待发货,刘同济,新疆维吾尔自治区伊犁哈萨克自治州霍尔果斯市,"[""订单A20250812163308673审核通过,请生成二维码给用户签合同"", ""137..."
160058,2025-08-12 16:38:38,A20250812163838760,8291,365天,5088.0,424.0,0.0,待支付,林佳佳,福建省厦门市湖里区,None
160154,2025-08-12 16:39:13,A20250812163912767,8289,365天,7836.0,653.0,0.0,待支付,周俊杰,福建省南平市武夷山市,None
161270,2025-08-12 16:43:18,A20250812164317862,8288,365天,6156.0,513.0,0.0,待审核,董辉,河北省沧州市运河区,"[""修改收货信息：将原收货信息 董先生 176****5038 河北省 沧州市 运河区 天成..."


# 用户画像

In [78]:
# 判断免人审的单可以用t_order_risk表中的risk json串中的is_vip=1和status=0
sql_xs = '''
select om.order_number, om.create_time, om.status, om.merchant_name,
tolog.delivery_province_name, tolog.delivery_city_name,
tmu.id_card_num
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result

from db_digua_business.t_order om
left join db_digua_business.t_order_logistics tolog on om.id=tolog.order_id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
where om.user_mobile is not null
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
and date_format(om.create_time, '%Y-%m-%d')>='2025-08-15'
and date_format(om.create_time, '%Y-%m-%d')<='2025-08-19'
and om.status not in (1, 13)
'''

df_xs = clean.query(sql_xs)
df_xs.shape

(11625, 15)

In [79]:
sql_risk = ''' -- risk等级
select trace_id, id_card as id_card_r, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 

from db_credit.risk
'''
df_risk = clean.query(sql_risk)

In [80]:
def drop_merchant(df):
    '''
    删除商家数据
    :param df: 传入带有商家的数据
    :return: 返回剔除了商家的数据
    '''
    # 剔除商家数据只保留自营租机业务数据
    df.drop(df[df['merchant_name'] == "深圳优优大数据科技有限公司"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "优优2店"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "小豚租（代收）"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "苏州蚁诺宝"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "租着用电脑数码"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "北京海鸟窝科技有限公司"].index, inplace=True)

    df.drop(df[df['merchant_name'] == "汇客好租"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "澄心优租"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "CPS渠道合作"].index, inplace=True)
    # df.drop(df[df['sku_attributes'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
    # 趣智数码  单
    df.drop(df[df['merchant_name'] == "趣智数码"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "格木木二奢名品"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "广州康基贸易有限公司"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "线下小店"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "乙辉数码"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "呱子笔记本电脑"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "南京聚格网络科技"].index, inplace=True)
    
    df.drop(df[df['merchant_name'] == "星晟数码"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "蘑菇时间"].index, inplace=True)

    df.drop(df[df['merchant_name'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
    return df

In [81]:
# 删除拒量数据
def drop_rejected_merchant(df):
    df.drop(df[df['merchant_name'] == "小蚂蚁租机"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "兴鑫兴通讯"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "人人享租"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "崇胜数码"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "喜卓灵租机"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "喜卓灵新租机"].index, inplace=True)
    if '机审通过件' in df.columns:
        df = df[~(df.tips.str.contains(r'策略241205') == True)]
        df = df[~(df.tips.str.contains('策略241212', regex=False)==True)]
        df = df[~(df.tips.str.contains('命中自有模型回捞策略', regex=False)==True)]
        df = df[~(df.tips.str.contains('回捞策略250330命中', regex=False)==True)]
    return df

In [91]:
# # 判断免人审的单可以用t_order_risk表中的risk json串中的is_vip=1和status=0
# df_ms = df[(df.is_vip=='1')&(df.status_result=='0')]
# df_ms2 = df2[(df2.is_vip=='1')&(df2.status_result=='0')]
df = df_xs.copy()
# 拼接df与df_risk,基于trace_id
df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left')
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
# 基于下单日期与id_card_num去重
df = df.drop_duplicates(subset=['下单日期', 'id_card_num'],keep="first")
# 去除商家订单
df = drop_merchant(df)

df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
# note 2025-02-24修改
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否拒量'] = np.where(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中') == True, 1, 0)
# 筛选出未命中前置拦截和机审强拒的订单
df = df[(df['是否前置拦截']==0)&(df['是否机审强拒']==0)]


df2 = df.copy()
# 删除拒量数据
df2 = drop_rejected_merchant(df2)
df2 = df2[df2.是否拒量==0]

# 筛选出信审订单
df_credit_examine = df[(df.is_vip=='0') &(df.status_result=='0')]
df_credit_examine2 = df2[((df2.is_vip=='0')&(df2.status_result=='0'))]
df_credit_examine2.shape

(120, 20)

In [92]:
# 筛选未命中发货需上锁的订单
df_credit_examine = df_credit_examine[~(df_credit_examine.tips.str.contains('发货需上锁', na=False))]
df_credit_examine2 = df_credit_examine2[~(df_credit_examine2.tips.str.contains('发货需上锁', na=False))]
df_credit_examine2.shape

(120, 20)

In [70]:

df_credit_examine2.groupby('下单日期').agg({"order_number": 'count'}).rename(columns={"order_number": "符合要求人数"}).reset_index()

,下单日期,符合要求人数
0,2025-08-15,38
1,2025-08-16,41
2,2025-08-17,26
3,2025-08-18,36
4,2025-08-19,28


In [ ]:
df_credit_examine.head()#[df_credit_examine['merchant_name']=='']

In [93]:
df_credit_examine2.groupby('merchant_name').agg({"order_number": 'count'}).rename(columns={"order_number": "符合要求人数"}).reset_index()

,merchant_name,符合要求人数
0,海南鸿丰商业保理有限公司,1
1,维客自营账户,119


In [ ]:
602
535
471
702
624
